In [1]:
#pip install finvizfinance

import pandas as pd
from finvizfinance.quote import finvizfinance
from datetime import datetime
import pathlib

import pandas_datareader.data as web
import numpy as np
from sklearn.linear_model import LinearRegression

from dateutil.relativedelta import relativedelta

import matplotlib.pyplot as plt

from IPython.display import clear_output



In [2]:
#!pip install dateutil.relativedelta

In [3]:
# NO EXECUTAR
# -----------

from finvizfinance.screener.overview import Overview
foverview = Overview()
#filters_dict = {'Exchange':'NYSE','Market Cap.':'+Mid (over $2bln)'}
filters_dict = {'Exchange':'NYSE','Price':'Over $10','Average Volume':'Over 100K'}
foverview.set_filter(filters_dict=filters_dict)
df = foverview.screener_view()



In [7]:
now = datetime.now()
#print(now.strftime("%Y-%m-%d"))

future_date_2months = now + relativedelta(months=2)
future_date_4months = now + relativedelta(months=4)
future_date_2months = future_date_2months.replace(day=1)
future_date_4months = future_date_4months.replace(day=1)

#print(future_date_2months)
#print(future_date_4months)


In [8]:

def plot_options_IV(param_ticker, param_expiry_date_month, param_expiry_date_year):
    #str_ticker='AAPL'

    ticker = web.YahooOptions(param_ticker)
    ticker.headers = {'User-Agent': 'Firefox'}
    #print(ticker.expiry_dates)

    # get call data
    df_calls = ticker.get_call_data(month = param_expiry_date_month , year = param_expiry_date_year)

    df_calls.to_csv('dataframe_opcions.csv')
    df_calls1 = pd.read_csv ('dataframe_opcions.csv')

    str_vto = df_calls1["Expiry"].values[0]

    # Plotting
    plt.figure(figsize=(12, 6))
    plt.plot(df_calls1['Strike'], df_calls1['IV'], label='IV')

    # Adding labels and title
    #plt.xlabel('Timestamp')
    #plt.ylabel('Close Price')
    plt.title('IV (Implied Volatility) - ' + param_ticker)

    # Ajustar una regressió lineal
    X = df_calls1[['Strike']]
    y = df_calls1['IV']

    regression_model = LinearRegression()
    regression_model.fit(X, y)

    # Preveure els valors de IV amb la regressió lineal
    y_pred = regression_model.predict(X)

    # Afegir la línia de regressió lineal
    plt.plot(df_calls1['Strike'], y_pred, label='Linear Regression', linestyle='--', color='red')

    # Seleccionar 20 valors de 'last' de manera equidistant
    selected_points = df_calls1.iloc[::len(df_calls1) // 10]

    # Afegir etiquetes 'last' per a cada punt seleccionat
    for index, row in selected_points.iterrows():
        last_value = row['Last']
        strike_value = row['Strike']
        plt.annotate(f'Last: {last_value:.2f}', xy=(strike_value, row['IV']), xytext=(5, 5),
                     textcoords='offset points', fontsize=8, color='blue')

    # Etiquetes i títol
    plt.xlabel('Strike')
    plt.ylabel('IV')
    plt.title('Gràfic de Strike vs. IV - ' + param_ticker + '(' + str_vto + ')')

    # Afegir llegenda
    plt.legend()

    # Mostrar la gràfica
    plt.show()
    

In [9]:
#URL = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
#ticker_list = pd.read_html(URL)[0]['Symbol'].tolist()

ticker_list = ['AAPL','MSFT','AMZN','META']

for str_ticker in ticker_list:
    #print(str_ticker)
    ticker = web.YahooOptions(str_ticker)
    ticker.headers = {'User-Agent': 'Firefox'}
    #print(exp.isoformat())

    for exp in ticker.expiry_dates:
        #print(exp.strftime("%Y-%m-%d"))
        if exp >= future_date_2months.date() and exp <= future_date_4months.date():
            exp_month = int(exp.strftime("%m"))
            exp_year = int(exp.strftime("%Y"))
            #print('Aquesta SIII: ' + exp.strftime("%Y-%m-%d") + ' month:' + 
            + ' year:' + str(exp_year))

            try:
                plot_options_IV(str_ticker, exp_month, exp_year)
            except ValueError:
                print("Error: " + str_ticker)      

            wait = input("Press Enter to continue.")
            clear_output(wait=False)
                
            
print('End')
    


SyntaxError: unmatched ')' (3600325514.py, line 18)